In [15]:
!pip install segmentation_models_pytorch

In [0]:
import os
import time  
from tqdm import tqdm  
import warnings
warnings.filterwarnings("ignore")
  
import pandas as pd
import numpy as np 
import random
from matplotlib import pyplot as plt
seed = 69
random.seed(seed)
np.random.seed(seed)   
os.environ["PYTHONHASHSEED"] = str(seed)
 

from sklearn.model_selection import train_test_split


import cv2
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.utils.data as data
from torch.utils.data import DataLoader, Dataset, sampler
from albumentations import (HorizontalFlip, ShiftScaleRotate, Normalize, Resize, Compose, GaussNoise)
from albumentations.pytorch import ToTensor
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data.sampler import WeightedRandomSampler

torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True 


import segmentation_models_pytorch as smp

from collections import Counter
from PIL import Image
import math
import seaborn as sns
from collections import defaultdict
from pathlib import Path
from tqdm import tqdm 

In [17]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive/')

Mounted at /content/drive/


In [0]:
path = '/content/drive/My Drive/project/data_main/'

# **Get Data**


In [0]:
def mask2rle(img):
    '''
    For an ouput numpy array
    1  == Mask, 
    0  == background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [0]:
def make_mask(row_id, df):
    '''
    Given a row index, 
    return a image id and mask of the image in the format of (256, 1600, 4)
    from the dataframe `df`
    '''
    fname = df.iloc[row_id].name
    labels = df.iloc[row_id][:4]
    masks = np.zeros((256, 1600, 4), dtype=np.float32) 
    for idx, label in enumerate(labels.values):
        if label is not np.nan:
            label = label.split(" ")
            positions = map(int, label[0::2])
            length = map(int, label[1::2])
            mask = np.zeros(256 * 1600, dtype=np.uint8)
            for pos, le in zip(positions, length):
                mask[pos:(pos + le)] = 1
            masks[:, :, idx] = mask.reshape(256, 1600, order='F')
    return fname, masks

In [0]:
def get_transforms(phase, mean, std):
    '''
    Augment the data with Horizontal flip 
    Transform the data with normalisation & convert to tensor. 
    '''
    list_transforms = []
    if phase == "train":
        list_transforms.extend(
            [
                HorizontalFlip(p=0.5), 
            ]
        )
    list_transforms.extend(
        [
            Normalize(mean=mean, std=std, p=1),
            ToTensor(),
        ]
    )
    list_trfms = Compose(list_transforms)
    return list_trfms

In [0]:
class TrainDataset(Dataset):
    '''
    Train dataset,
    This is made in the form to be inputed in dataloader
    Hence, it contains __getitem__ and __len__
    '''
    def __init__(self, df, data_folder, mean, std, phase):
        self.df = df
        self.root = data_folder
        self.mean = mean
        self.std = std
        self.phase = phase
        self.transforms = get_transforms(phase, mean, std)
        self.fnames = self.df.index.tolist()
    def __getitem__(self, idx):
        image_id, mask = make_mask(idx, self.df)
        image_path = os.path.join(self.root, "train_images",  image_id)
        img = cv2.imread(image_path)
        augmented = self.transforms(image=img, mask=mask)
        img = augmented['image']
        mask = augmented['mask'] # 1x256x1600x4
        mask = mask[0].permute(2, 0, 1) # 1x4x256x1600
        return img, mask
    def __len__(self):
        return len(self.fnames)

In [0]:
def class_flag(x):
    nan = 'nan'
    if x[4] != nan:
        return 4
    elif x[3] != nan:
        return 3
    elif x[2] != nan:
        return 2
    elif x[1] != nan  :
        print(x)
        return 1  
    else:
      return 0 

def provider(data_folder,df_path,phase,mean=None,std=None,batch_size=8,num_workers=4,): 
    '''Returns dataloader for the model training'''
    df = pd.read_csv(df_path) 
    ### Get the weights for class imbalance 
    class_dict = defaultdict(int)
    kind_class_dict = defaultdict(int)

    no_defects_num = 0
    defects_num = 0

    for col in range(0, len(df), 4):
        img_names = [str(i).split("_")[0] for i in df.iloc[col:col+4, 0].values]
        if not (img_names[0] == img_names[1] == img_names[2] == img_names[3]):
            raise ValueError
            
        labels = df.iloc[col:col+4, 1]
        if labels.isna().all():
            no_defects_num += 1
        else:
            defects_num += 1
        
        kind_class_dict[sum(labels.isna().values == False)] += 1
            
        for idx, label in enumerate(labels.isna().values.tolist()):  
            if label == False:
                class_dict[idx+1] += 1
        if sum(labels.isna().values.tolist()) == 4: 
          class_dict[5] += 1                 
    df['ImageId'], df['ClassId'] = zip(*df['ImageId_ClassId'].str.split('_'))
    df['ClassId'] = df['ClassId'].astype(int)
    df = df.pivot(index='ImageId',columns='ClassId',values='EncodedPixels')
    df['defects'] = df.count(axis=1)
    train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["defects"], random_state=69)
    df = train_df if phase == "train" else val_df
    image_dataset = TrainDataset(df, data_folder, mean, std, phase)
    df2 = df.reset_index()
    df2[1] = df2[1].astype(str)
    df2[2] = df2[2].astype(str)
    df2[3] = df2[3].astype(str)
    df2[4] = df2[1].astype(str)
    df2['class_id'] = df2.apply(class_flag, axis = 1)
    weight = 1/np.array(list(class_dict.values()))
    samples_weight = np.array([weight[t] for t in list(df2['class_id'] - 1)])
    samples_weight = torch.from_numpy(samples_weight)
    samples_weigth = samples_weight.double()
    sampler = WeightedRandomSampler(samples_weight, len(samples_weight)) 
    dataloader = DataLoader(
        image_dataset,
        sampler=sampler,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=True,
        shuffle=False,   
    )
    return dataloader 


# **Model**

In [0]:
model = smp.Unet("resnet18", encoder_weights=None , classes=4, activation='softmax')

# Train & Validating the data

In [0]:
sample_submission_path = path + 'sample_submission.csv'
train_df_path = path + 'train.csv'
data_folder = path
test_data_folder = path + "test_images"

In [0]:
def dice_loss(input_, target):
    smooth = 1.
    iflat = input_.view(-1)
    tflat = target.view(-1)
    intersection = (iflat * tflat).sum()
    
    return 1 - ((2. * intersection + smooth) /
              (iflat.sum() + tflat.sum() + smooth))

In [0]:
class Trainer(object):
    '''This class takes care of training and validation of our model'''
    def __init__(self, model):
        self.num_workers = 6
        self.batch_size = {"train": 4, "val": 4}
        self.accumulation_steps = 32 // self.batch_size['train']
        self.lr = 5e-4
        self.num_epochs = 20
        self.best_loss = float("inf")
        self.phases = ["train", "val"]
        self.device = torch.device("cuda:0")
        torch.set_default_tensor_type("torch.cuda.FloatTensor")
        self.net = model
        self.criterion = dice_loss
        self.optimizer = optim.Adam(self.net.parameters(), lr=self.lr)
        self.scheduler = ReduceLROnPlateau(self.optimizer, mode="min", patience=3, verbose=True)
        self.net = self.net.to(self.device)
        cudnn.benchmark = True
        self.dataloaders = { phase: provider(
        data_folder=data_folder,
        df_path=train_df_path,
        phase=phase,
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225),
        batch_size=self.batch_size[phase],
        num_workers=self.num_workers, ) for phase in self.phases} 
        self.losses = {phase: [] for phase in self.phases}
    def forward(self, images, targets):
        images = images.to(self.device)
        masks = targets.to(self.device)
        outputs = self.net(images)
        loss = self.criterion(outputs, masks)
        return loss, outputs
    def iterate(self, epoch, phase):
        start = time.strftime("%H:%M:%S")
        print(f"Starting epoch: {epoch} | Phae: {phase} | Start Time : {start}")
        batch_size = self.batch_size[phase]
        self.net.train(phase == "train")
        dataloader = self.dataloaders[phase]
        running_loss = 0.0
        total_batches = len(dataloader)
        self.optimizer.zero_grad()
        for itr, batch in enumerate(dataloader): 
            images, targets = batch
            loss, outputs = self.forward(images, targets)
            loss = loss / self.accumulation_steps
            if phase == "train":
                loss.backward()
                if (itr + 1 ) % self.accumulation_steps == 0:
                    self.optimizer.step()
                    self.optimizer.zero_grad()
            running_loss += loss.item()
            outputs = outputs.detach().cpu()
        epoch_loss = (running_loss * self.accumulation_steps) / total_batches
        self.losses[phase].append(epoch_loss)
        torch.cuda.empty_cache()
        return epoch_loss
    def start(self):
        for epoch in range(self.num_epochs):
            self.iterate(epoch, "train")
            state = {
                "epoch": epoch,
                "best_loss": self.best_loss,
                "state_dict": self.net.state_dict(),
                "optimizer": self.optimizer.state_dict(),
            }
            with torch.no_grad():
                val_loss = self.iterate(epoch, "val")
                self.scheduler.step(val_loss)
            if val_loss < self.best_loss:
                state["best_loss"] = self.best_loss = val_loss
                torch.save(state, "./model.pth")
            print(f"For epoch: {epoch} | Loss is: {val_loss}")
        print(f"For UNET Model, without Pretrain weight and class imbalance for 20 epoches , the best loss is: {self.best_loss}")    
    

In [0]:
model_trainer = Trainer(model)
model_trainer.start()

Starting epoch: 0 | Phae: train | Start Time : 12:12:10
Starting epoch: 0 | Phae: val | Start Time : 12:40:22
For epoch: 0 | Loss is: 0.9913508593562298
Starting epoch: 1 | Phae: train | Start Time : 12:43:07
Starting epoch: 1 | Phae: val | Start Time : 13:10:52
For epoch: 1 | Loss is: 1.0012135786168717
Starting epoch: 2 | Phae: train | Start Time : 13:13:15
Starting epoch: 2 | Phae: val | Start Time : 13:41:04
For epoch: 2 | Loss is: 1.0042069164861367
Starting epoch: 3 | Phae: train | Start Time : 13:43:27
Starting epoch: 3 | Phae: val | Start Time : 14:11:18
For epoch: 3 | Loss is: 0.9655768785074899
Starting epoch: 4 | Phae: train | Start Time : 14:13:49
Starting epoch: 4 | Phae: val | Start Time : 14:41:35
For epoch: 4 | Loss is: 1.014053603897792
Starting epoch: 5 | Phae: train | Start Time : 14:43:58
Starting epoch: 5 | Phae: val | Start Time : 15:11:46
For epoch: 5 | Loss is: 0.989211077625687
Starting epoch: 6 | Phae: train | Start Time : 15:14:09
Starting epoch: 6 | Phae: va